In [ ]:
import os
import cv2
import numpy as np
import face_recognition

### This notebook grab the video and extract the frames having faces only

- The face_preprocessing function takes the videos from input_path and saves the frames in Output path 
- It creates filename same as video name and creates two sub folders in it namely face and flow .
- It only populates the face folder 

- The expected directory structure for the function is shown below

<img src='directory_flow1.png'>

- The expected output directory structure
<img src='directory_flow2.png'>



In [ ]:
os.getcwd()

In [ ]:
def preprocessing(input_path,output_path):
    files = os.listdir(input_path)
    files.sort()
    print(f"{len(files)} files found")
    videos = []
    for file in files:
        path = os.path.join(input_path + '/'+file)
        videos.append(path)


    videoCount = 0
    for video in videos:
        print('processing....' + video)
        videoCount+=1

        os.mkdir(os.path.join(output_path + '/'+ video.split('/')[-1]))
        outputPath = os.path.join(output_path + '/'+ video.split('/')[-1] )
        #print(outputPath)
        os.mkdir(os.path.join(outputPath+'/face'))
        os.mkdir(os.path.join(outputPath+'/flow'))
        facePath = os.path.join(outputPath+'/face')
        flowPath = os.path.join(outputPath+'/flow')


        # Extracting face cropped video
        cap = cv2.VideoCapture(video)


        #read frames until a face is encountered
        face_image = None
        while(True):
            ret, frame = cap.read()
            if(ret == True):
                face_image = face_recognition.face_locations(frame)
                if(len(face_image) > 0):
                    break


        top,right,bottom,left = face_image[0]

        boxleft = left -  30 if left-30 > 0 else left 
        boxright = right + 30 if right+30 < frame.shape[1]  else right
        boxtop = top - 30 if top-30 > 0 else top
        boxbottom = bottom + 30 if bottom+30 < frame.shape[0] else bottom

        size = (640,360)

        cv2.imwrite(
            facePath+ '/' + '0000' +'.jpg',
            cv2.resize(frame[boxtop:boxbottom,boxleft:boxright,:],size)
        )

        frameCount = 1

        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret == True:
                try:
                    face_image = face_recognition.face_locations(frame)
                    if(len(face_image) > 0):
                        top,right,bottom,left = face_image[0]

                        if(top < boxtop or bottom > boxbottom or right > boxright or left < boxleft):
                            # update box
                            boxtop = top - 30;
                            boxright = right + 30
                            boxleft = left - 30
                            boxbottom = bottom + 30


                        paddedCount = '%04d' % frameCount
                        cv2.imwrite(
                            facePath+'/'+ paddedCount +'.jpg',
                            cv2.resize(frame[boxtop:boxbottom,boxleft:boxright,:],size)            
                        )
            
                        frameCount+=1
                        if frameCount == 120:
                            break
                except:
                    print("Frame skipped")
                    pass

            else:
                break
    print(f"Processed {videoCount} videos")


    
    

In [ ]:
#Create output directory manually
#os.mkdir('Output')
input_path = os.path.join(os.getcwd() + '/dataset/test/fake')
output_path = os.path.join(os.getcwd()+'/Output')

In [ ]:
preprocessing(input_path,output_path)